In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def morph_tri(im1, im2, im1_pts, im2_pts, warp_frac, dissolve_frac):
  # TODO: Your code here
  # Tips: use Delaunay() function to get Delaunay triangulation;
  # Tips: use tri.find_simplex(pts) to find the triangulation index that pts locates in.
    frame_num = warp_frac.shape[0] 
    im1_morphed = np.zeros((frame_num, im1.shape[0], im1.shape[1], im1.shape[2]))
    im2_morphed = im1_morphed.copy()
    morphed_im = im1_morphed.copy()
    
    for f in range(frame_num):
        intermediate_im = (1-warp_frac[f]) * im1_pts + warp_frac[f] * im2_pts
        delaunay_obj = spatial.Delaunay(intermediate_im)
        intermediate_simplex = delaunay_obj.simplices
        
        for y in range(im1.shape[0]):
            for x in range(im1.shape[1]):
                simplex_idx = delaunay_obj.find_simplex(np.array([x,y]))
                
                matrix = np.transpose(intermediate_im[intermediate_simplex[simplex_idx]])
                matrix = np.vstack((matrix,np.ones((1,3))))
                barycentric_coor = np.dot(np.linalg.inv(matrix), np.array([[x],[y],[1]]))
                
                matrix_im1 = np.transpose(im1_pts[intermediate_simplex[simplex_idx]])
                matrix_im1 = np.vstack((matrix_im1,np.ones((1,3))))

                matrix_im2 = np.transpose(im2_pts[intermediate_simplex[simplex_idx]])
                matrix_im2 = np.vstack((matrix_im2,np.ones((1,3))))
                
                corr1_coor = np.dot(matrix_im1, barycentric_coor)
                corr2_coor = np.dot(matrix_im2, barycentric_coor)
                
                y1 = int(corr1_coor[0])
                x1 = int(corr1_coor[1])
                
                y2 = int(corr2_coor[0])
                x2 = int(corr2_coor[1])

                
                im1_morphed[f, y, x, 0] = im1[int(x1), int(y1), 0]
                im1_morphed[f, y, x, 1] = im1[int(x1), int(y1), 1]
                im1_morphed[f, y, x, 2] = im1[int(x1), int(y1), 2]
                
                im2_morphed[f, y, x, 0] = im2[int(x2), int(y2), 0]
                im2_morphed[f, y, x, 1] = im2[int(x2), int(y2), 1]
                im2_morphed[f, y, x, 2] = im2[int(x2), int(y2), 2]
                
        morphed_im[f, :, :, :] = (1 - dissolve_frac[f]) * im1_morphed[f, :, :, :] + dissolve_frac[f] * im2_morphed[f, :, :, :]


    # discretization
    np.clip(morphed_im, 0, 255, out = morphed_im)

    
    return morphed_im.astype('uint8')
    

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from numpy.linalg import inv
from PIL import Image as im
import matplotlib.image as mpimg
from scipy.spatial import Delaunay
import imageio

In [4]:
# lst = [[0,0],[0,0],[150,0], [150,0], [0,150], [0,150], [300,0], [300,0], [0,300], [0,300], [300,300],[300,300], [150,300],[150,300],[300,150],[300,150],[115.26612903225806, 219.49193548387095], [130.50806451612908, 187.55645161290317], [160.50806451612902, 222.1532258064516], [175.74999999999994, 190.21774193548384], [107.28225806451611, 248.76612903225805], [118.53225806451616, 208.84677419354836], [157.8467741935484, 247.43548387096772], [189.05645161290323, 211.508064516129], [135.22580645161287, 260.741935483871], [150.46774193548384, 230.13709677419348], [135.22580645161287, 239.4516129032258], [153.12903225806446, 204.85483870967738], [133.89516129032256, 219.49193548387095], [154.45967741935482, 188.88709677419354], [120.58870967741935, 188.88709677419354], [138.49193548387092, 160.94354838709677], [157.8467741935484, 186.22580645161287], [171.75806451612897, 163.60483870967738], [129.9032258064516, 276.7096774193548], [153.12903225806446, 263.40322580645153], [78.00806451612901, 264.73387096774195], [105.22580645161287, 242.1129032258064], [51.39516129032258, 242.11290322580643], [77.28225806451616, 212.8387096774193], [185.79032258064518, 267.39516129032256], [195.70967741935482, 246.10483870967735], [223.0483870967742, 243.44354838709674], [215.6693548387097, 218.16129032258058], [48.733870967741936, 226.14516129032256], [70.62903225806457, 190.21774193548384], [19.459677419354833, 208.84677419354836], [45.346774193548356, 172.91935483870964], [14.137096774193537, 179.57258064516128], [45.346774193548356, 148.96774193548384], [22.120967741935473, 147.63709677419354], [42.685483870967744, 126.34677419354836], [15.467741935483858, 114.37096774193549], [42.685483870967744, 94.41129032258067], [30.104838709677423, 74.4516129032258], [53.330645161290306, 59.814516129032256], [85.99193548387096, 47.83870967741933], [86.59677419354836, 33.20161290322585], [147.2016129032258, 35.86290322580646], [123.85483870967738, 18.564516129032313], [203.08870967741933, 41.185483870967744], [179.74193548387092, 11.911290322580669], [240.34677419354836, 61.14516129032256], [234.2983870967742, 29.209677419354875], [266.9596774193548, 86.42741935483872], [260.91129032258056, 81.10483870967744], [278.93548387096774, 127.67741935483872], [264.90322580645153, 122.35483870967738], [272.2822580645161, 163.60483870967738], [258.24999999999994, 143.64516129032256], [285.58870967741933, 188.88709677419354], [252.92741935483872, 158.2822580645161], [268.2903225806452, 214.16935483870967], [247.60483870967738, 178.24193548387092], [245.6693548387097, 232.79838709677418], [231.63709677419348, 199.5322580645161], [78.00806451612901, 180.9032258064516], [99.90322580645164, 139.6532258064516], [115.26612903225806, 180.9032258064516], [130.50806451612908, 143.64516129032256], [171.1532258064516, 180.9032258064516], [175.74999999999994, 146.30645161290317], [211.07258064516128, 182.2338709677419], [211.67741935483872, 143.64516129032256], [96.63709677419352, 170.25806451612902], [114.54032258064518, 135.6612903225806], [191.11290322580646, 172.91935483870964], [194.37903225806446, 135.6612903225806], [96.63709677419352, 186.22580645161287], [193.0483870967742, 150.29838709677415], [67.36290322580645, 158.2822580645161], [89.25806451612908, 125.01612903225805], [83.33064516129032, 146.30645161290323], [103.89516129032262, 113.04032258064512], [109.94354838709675, 152.95967741935482], [133.1693548387097, 123.68548387096774], [171.1532258064516, 162.27419354838707], [177.0806451612903, 122.35483870967738], [200.42741935483872, 154.29032258064512], [201.03225806451616, 115.7016129032258], [223.0483870967742, 162.27419354838707], [226.31451612903226, 130.33870967741933], [63.37096774193547, 262.0725806451613], [85.2661290322581, 258.08064516129025], [209.74193548387098, 270.0564516129032], [201.03225806451616, 256.74999999999994], [38.088709677419345, 268.72580645161287], [66.6370967741936, 286.024193548387], [235.02419354838707, 268.72580645161287], [219.66129032258067, 287.3548387096773], [156.51612903225805, 90.41935483870964], [158.4516129032258, 58.48387096774195], [101.95967741935483, 101.06451612903226], [102.56451612903226, 79.77419354838707], [54.05645161290322, 129.00806451612902], [81.27419354838713, 110.37903225806451], [225.70967741935482, 89.08870967741933], [202.3629032258064, 75.78225806451616], [252.32258064516128, 144.97580645161287], [224.9838709677419, 117.0322580645161]]
# lst= [[0,0],[0,0],[150,0], [150,0], [0,150], [0,150], [300,0], [300,0], [0,300], [0,300], [300,300],[300,300], [150,300],[150,300],[300,150],[300,150],[112.6048387096774, 219.49193548387095], [131.83870967741933, 186.22580645161287], [163.1693548387097, 216.8306451612903], [178.41129032258067, 182.2338709677419], [111.27419354838709, 246.1048387096774], [117.2016129032258, 210.17741935483866], [156.51612903225805, 248.76612903225805], [189.05645161290323, 211.508064516129], [132.56451612903226, 258.0806451612903], [153.12903225806446, 230.13709677419348], [132.56451612903226, 235.45967741935482], [147.80645161290323, 202.1935483870967], [132.56451612903226, 282.0322580645161], [154.45967741935482, 259.4112903225806], [78.00806451612901, 262.0725806451613], [98.57258064516128, 231.46774193548382], [50.06451612903224, 232.79838709677418], [70.62903225806457, 191.54838709677415], [10.145161290322577, 192.8790322580645], [44.01612903225811, 164.9354838709677], [18.129032258064512, 162.27419354838707], [45.346774193548356, 131.66935483870964], [26.11290322580645, 66.46774193548384], [55.99193548387103, 41.185483870967744], [113.93548387096773, 37.19354838709677], [129.17741935483872, 21.225806451612925], [223.0483870967742, 50.5], [254.25806451612897, 57.153225806451644], [281.59677419354836, 122.35483870967738], [268.8951612903225, 109.0483870967742], [269.6209677419355, 164.93548387096774], [255.58870967741933, 136.99193548387098], [281.59677419354836, 183.56451612903226], [250.266129032258, 154.29032258064512], [268.2903225806452, 214.16935483870967], [248.93548387096774, 175.5806451612903], [233.69354838709677, 234.12903225806448], [227.6451612903225, 199.5322580645161], [205.75, 255.41935483870967], [213.00806451612897, 228.80645161290317], [137.88709677419354, 219.49193548387095], [151.7983870967742, 187.55645161290317], [70.02419354838709, 175.5806451612903], [99.90322580645164, 142.31451612903226], [116.59677419354837, 179.57258064516128], [138.49193548387092, 146.30645161290317], [173.81451612903226, 180.9032258064516], [174.4193548387097, 147.63709677419354], [211.07258064516128, 183.56451612903226], [211.67741935483872, 143.64516129032256], [100.6290322580645, 186.22580645161287], [115.87096774193543, 147.63709677419354], [191.11290322580646, 190.21774193548384], [197.04032258064518, 155.62096774193543], [92.64516129032258, 170.25806451612902], [193.0483870967742, 136.99193548387098], [62.04032258064514, 158.2822580645161], [85.2661290322581, 123.68548387096774], [105.95161290322578, 150.29838709677415], [131.83870967741933, 118.36290322580646], [183.1290322580645, 158.2822580645161], [171.75806451612897, 123.68548387096774], [224.3790322580645, 168.92741935483872], [220.99193548387092, 129.00806451612902], [76.6774193548387, 150.29838709677415], [107.8870967741936, 111.70967741935482], [201.75806451612902, 151.6290322580645], [197.04032258064518, 111.70967741935482], [55.38709677419355, 115.7016129032258], [89.25806451612908, 85.09677419354841], [152.52419354838707, 93.0806451612903], [154.45967741935482, 61.14516129032256], [247.0, 99.73387096774195], [210.34677419354836, 82.43548387096774]]
lst = [[0,0],[0,0],[150,0], [150,0], [0,150], [0,150], [300,0], [300,0], [0,300], [0,300], [300,300],[300,300], [150,300],[150,300],[300,150],[300,150], [80.66935483870968, 179.57258064516128], [101.2338709677419, 150.29838709677415], [115.26612903225806, 178.24193548387098], [139.82258064516128, 162.27419354838707], [171.1532258064516, 182.2338709677419], [186.39516129032262, 160.94354838709677], [213.73387096774195, 182.2338709677419], [224.9838709677419, 146.30645161290317], [97.96774193548386, 184.89516129032256], [111.87903225806457, 166.26612903225805], [191.11290322580646, 187.55645161290323], [207.68548387096774, 164.9354838709677], [95.30645161290322, 168.92741935483872], [115.87096774193543, 140.9838709677419], [188.4516129032258, 174.25], [203.69354838709677, 140.9838709677419], [115.26612903225806, 216.8306451612903], [150.46774193548384, 203.52419354838707], [161.83870967741933, 218.1612903225806], [178.41129032258067, 202.1935483870967], [136.55645161290323, 222.1532258064516], [163.77419354838713, 218.16129032258058], [112.6048387096774, 247.43548387096772], [145.14516129032262, 232.79838709677415], [157.8467741935484, 252.75806451612902], [178.41129032258067, 230.13709677419348], [131.2338709677419, 256.75], [165.10483870967738, 234.12903225806446], [136.55645161290323, 239.4516129032258], [162.44354838709677, 222.15322580645156], [131.2338709677419, 278.0403225806451], [162.44354838709677, 248.766129032258], [79.33870967741935, 263.4032258064516], [106.55645161290323, 238.12096774193543], [59.3790322580645, 251.4274193548387], [73.29032258064518, 219.49193548387092], [46.07258064516127, 227.4758064516129], [57.32258064516128, 166.26612903225805], [14.137096774193537, 196.8709677419355], [46.67741935483872, 83.76612903225805], [35.427419354838705, 65.13709677419354], [48.00806451612908, 3.927419354838719], [149.86290322580646, 31.870967741935488], [153.12903225806446, 62.475806451612925], [236.35483870967744, 55.82258064516128], [260.91129032258056, 11.911290322580669], [278.93548387096774, 114.37096774193549], [246.27419354838702, 111.70967741935482], [273.61290322580646, 160.94354838709677], [256.9193548387097, 144.97580645161287], [282.9274193548387, 191.54838709677415], [254.25806451612897, 174.24999999999994], [253.6532258064516, 227.4758064516129], [247.60483870967738, 196.87096774193543], [217.72580645161293, 254.08870967741933], [224.9838709677419, 211.508064516129], [183.1290322580645, 267.39516129032256], [206.35483870967738, 234.12903225806446]]
im1_pts = [lst[i] for i in range(len(lst)) if i%2 == 0]
im2_pts = [lst[i] for i in range(len(lst)) if i%2 != 0]
im1_pts = np.array(im1_pts)
im2_pts = np.array(im2_pts)

im1 = np.array(Image.open('baby.jpg'))
im2 = np.array(Image.open('cat.jpg'))

warp_frac=np.arange(0,1,1/50)
dissolve_frac = warp_frac

In [5]:
morphed_im=morph_tri(im1, im2, im1_pts, im2_pts, warp_frac, dissolve_frac)
imgs=[]
for i in range(0,warp_frac.shape[0]):
    imgs.append(morphed_im[i, :, :, :])
imageio.mimsave('./output.gif', res_list)